### Transformaciones: Para este MVP no necesitas perfección, ¡necesitas rapidez! ⏩ Vas a hacer estas, y solo estas, transformaciones a los datos:

- Algunos campos, como belongs_to_collection, production_companies y otros (ver diccionario de datos) están anidados, esto es o bien tienen un diccionario o una lista como valores en cada fila, ¡deberán desanidarlos para poder y unirlos al dataset de nuevo hacer alguna de las consultas de la API! O bien buscar la manera de acceder a esos datos sin desanidarlos.

- Los valores nulos de los campos revenue, budget deben ser rellenados por el número 0.

- Los valores nulos del campo release date deben eliminarse.

- De haber fechas, deberán tener el formato AAAA-mm-dd, además deberán crear la columna release_year donde extraerán el año de la fecha de estreno.

- Crear la columna con el retorno de inversión, llamada return con los campos revenue y budget, dividiendo estas dos últimas revenue / budget, - cuando no hay datos disponibles para calcularlo, deberá tomar el valor 0.

- Eliminar las columnas que no serán utilizadas, video,imdb_id,adult,original_title,poster_path y homepage.

Nota: Algunos campos, como belongs_to_collection, production_companies y otros (ver diccionario de datos) están anidados, esto es o bien tienen un diccionario o una lista como valores en cada fila.

Deberán desanidarlos para poder unirlos al dataset de nuevo y hacer alguna de las consultas de la API!. 
O bien buscar la manera de acceder a esos datos sin desanidarlos.



In [85]:
import pandas as pd
import numpy as np
import json
import re


In [86]:
# cargar dataset
df=pd.read_csv('movies_dataset.csv')

C:\Users\TheFercho\AppData\Local\Temp\ipykernel_9928\3955605885.py:2: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv('movies_dataset.csv')


In [87]:
df.shape

(45466, 24)

#### Verificamos si las columnas son categoricas o numericas para luego transformarlas en el caso que sea necesario.

In [88]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  45466 non-null  object 
 1   belongs_to_collection  4494 non-null   object 
 2   budget                 45466 non-null  object 
 3   genres                 45466 non-null  object 
 4   homepage               7782 non-null   object 
 5   id                     45466 non-null  object 
 6   imdb_id                45449 non-null  object 
 7   original_language      45455 non-null  object 
 8   original_title         45466 non-null  object 
 9   overview               44512 non-null  object 
 10  popularity             45461 non-null  object 
 11  poster_path            45080 non-null  object 
 12  production_companies   45463 non-null  object 
 13  production_countries   45463 non-null  object 
 14  release_date           45379 non-null  object 
 15  re

In [89]:
df.loc[df['id']=='1997-08-20']

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
19730,- Written by Ørnås,0.065736,/ff9qCepilowshEtG2GYWwzt2bs4.jpg,"[{'name': 'Carousel Productions', 'id': 11176}...","[{'iso_3166_1': 'CA', 'name': 'Canada'}, {'iso...",1997-08-20,0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,...,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [90]:
# Filtrar los registros que no son enteros en la columna 'id'
registros_no_enteros = df.loc[~df['id'].str.isdigit()]
registros_no_enteros
#En este caso, se utiliza la función str.isdigit() para verificar si cada valor de la columna 'id' 
# es un número entero. Luego, se nega el resultado utilizando el operador ~ para obtener los registros 
# que no son enteros. El resultado se asigna a la variable registros_no_enteros, y finalmente se imprime.

#Esto te proporcionará los registros que no son números enteros en la columna 'id', 
# los cuales podrías modificar o convertir a tipo integer según tus necesidades.

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
19730,- Written by Ørnås,0.065736,/ff9qCepilowshEtG2GYWwzt2bs4.jpg,"[{'name': 'Carousel Productions', 'id': 11176}...","[{'iso_3166_1': 'CA', 'name': 'Canada'}, {'iso...",1997-08-20,0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,...,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29503,Rune Balot goes to a casino connected to the ...,1.931659,/zV8bHuSL6WXoD6FWogP9j4x80bL.jpg,"[{'name': 'Aniplex', 'id': 2883}, {'name': 'Go...","[{'iso_3166_1': 'US', 'name': 'United States o...",2012-09-29,0,68.0,"[{'iso_639_1': 'ja', 'name': '日本語'}]",Released,...,12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
35587,Avalanche Sharks tells the story of a bikini ...,2.185485,/zaSf5OG7V8X8gqFvly88zDdRm46.jpg,"[{'name': 'Odyssey Media', 'id': 17161}, {'nam...","[{'iso_3166_1': 'CA', 'name': 'Canada'}]",2014-01-01,0,82.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,...,22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [91]:
# Eliminar la fila donde la columna 'id' tiene el valor '1997-08-20'
df = df.drop(df[df['id'] == '1997-08-20'].index)

In [92]:
# Obtener los índices de los registros que no son enteros en la columna 'id'
indices_no_enteros = df[~df['id'].str.isdigit()].index

# Eliminar los registros que no son enteros
df = df.drop(indices_no_enteros)

#En este caso, se utiliza el método ~ junto con str.isdigit() para obtener los índices de las filas 
# que no son números enteros en la columna 'id'. Estos índices se asignan a la variable indices_no_enteros. 
# Luego, se utiliza el método drop junto con los índices para eliminar las filas correspondientes. 
# El resultado se asigna nuevamente al DataFrame df

In [93]:
df.id.astype(int)

0           862
1          8844
2         15602
3         31357
4         11862
          ...  
45461    439050
45462    111109
45463     67758
45464    227506
45465    461257
Name: id, Length: 45463, dtype: int32

#### - Eliminar las columnas que no serán utilizadas, video,imdb_id,adult,original_title,poster_path y homepage.

In [94]:
df.drop(['video','imdb_id','adult','original_title','poster_path','homepage'],axis=1,inplace=True)
df.columns

Index(['belongs_to_collection', 'budget', 'genres', 'id', 'original_language',
       'overview', 'popularity', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'vote_average',
       'vote_count'],
      dtype='object')

#### - Los valores nulos de los campos revenue, budget deben ser rellenados por el número 0.

In [95]:
df['revenue'].fillna(0,inplace=True)

In [96]:
df['budget'].fillna(0,inplace=True)

In [97]:
# verificamos valores nulos en general
df.isnull().sum()

belongs_to_collection    40972
budget                       0
genres                       0
id                           0
original_language           11
overview                   954
popularity                   3
production_companies         3
production_countries         3
release_date                87
revenue                      0
runtime                    260
spoken_languages             3
status                      84
tagline                  25051
title                        3
vote_average                 3
vote_count                   3
dtype: int64

In [98]:
# Mostrar los registros repetidos de la columna 'id'
duplicados = df[df.duplicated(['id'], keep=False)] # El parámetro keep=False se utiliza para marcar todos 
# los duplicados como True, de modo que se muestren todos los registros duplicados.
duplicados.head()

,belongs_to_collection,budget,genres,id,original_language,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
676,NaN,0,"[{'id': 18, 'name': 'Drama'}, {'id': 10749, 'n...",105045,de,"East-Berlin, 1961, shortly after the erection ...",0.122178,"[{'name': 'Studio Babelsberg', 'id': 264}, {'n...","[{'iso_3166_1': 'DE', 'name': 'Germany'}]",1995-02-16,0.0,115.0,"[{'iso_639_1': 'de', 'name': 'Deutsch'}]",Released,"A love, a hope, a wall.",The Promise,5.0,1.0
838,NaN,0,"[{'id': 18, 'name': 'Drama'}]",132641,ja,"Ten years into a marriage, the wife is disappo...",0.096079,"[{'name': 'Toho Company', 'id': 882}]","[{'iso_3166_1': 'JP', 'name': 'Japan'}]",1953-04-29,0.0,89.0,"[{'iso_639_1': 'ja', 'name': '日本語'}]",Released,NaN,Wife,0.0,0.0
949,NaN,4,"[{'id': 18, 'name': 'Drama'}, {'id': 10749, 'n...",22649,en,British nurse Catherine Barkley (Helen Hayes) ...,1.914697,"[{'name': 'Paramount Pictures', 'id': 4}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1932-12-08,25.0,89.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Every woman who has loved will understand,A Farewell to Arms,6.2,29.0
1465,NaN,0,"[{'id': 18, 'name': 'Drama'}, {'id': 10749, 'n...",105045,de,"East-Berlin, 1961, shortly after the erection ...",0.122178,"[{'name': 'Studio Babelsberg', 'id': 264}, {'n...","[{'iso_3166_1': 'DE', 'name': 'Germany'}]",1995-02-16,0.0,115.0,"[{'iso_639_1': 'de', 'name': 'Deutsch'}]",Released,"A love, a hope, a wall.",The Promise,5.0,1.0
2564,NaN,0,"[{'id': 99, 'name': 'Documentary'}]",84198,en,"Using personal stories, this powerful document...",0.501046,[],"[{'iso_3166_1': 'US', 'name': 'United States o...",2012-03-22,0.0,84.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,One Nation. Underfed.,A Place at the Table,6.9,7.0


In [99]:
df.iloc[45265]

belongs_to_collection                                                  NaN
budget                                                             5402000
genres                                      [{'id': 35, 'name': 'Comedy'}]
id                                                                  277839
original_language                                                       fr
overview                 Franky and Krimo dream of leaving the grey gri...
popularity                                                        5.613875
production_companies            [{'name': 'Mandarin Cinéma', 'id': 57294}]
production_countries              [{'iso_3166_1': 'FR', 'name': 'France'}]
release_date                                                    2016-02-24
revenue                                                                0.0
runtime                                                              100.0
spoken_languages                 [{'iso_639_1': 'fr', 'name': 'Français'}]
status                   

In [100]:
# Verificamos nulos de revenue y budget para reemplazamos por valor 0
df[['revenue','budget']].isnull().sum()

revenue    0
budget     0
dtype: int64

#### - Crear la columna con el retorno de inversión, llamada return con los campos revenue y budget, dividiendo estas dos últimas revenue / budget, cuando no hay datos disponibles para calcularlo, deberá tomar el valor 0.

In [101]:
# verificamos la fila en donde se encuetra el dato errado => [] y lo reemplazamos por 0
# Armar luego una funcion para automatizarlo
df.iloc[19714]=0
df.iloc[29472]=0
df.iloc[35543]=0

In [102]:
df['revenue'].head(2)

0    373554033.0
1    262797249.0
Name: revenue, dtype: float64

In [103]:
# Pasamos los valores de la col 'budget' string a float. Tenemos 2 formas:
#df[['budget']].astype(float)
#df[['revenue']].astype(float)
#pd.to_numeric(df['budget'], downcast='float')
#pd.to_numeric(df['revenue'], downcast='float')

In [104]:
df['revenue']=df['revenue'].astype(float)

In [105]:
df['budget']=df['budget'].astype(float)

#### - Creamos la columna 'return' con la division de las columnas 'budget' y 'revenue'

In [106]:

df['return']=df['revenue'] / df['budget']

#### - Los valores nulos del campo 'release_date' deben eliminarse.

In [107]:
#Los valores nulos del campo release date deben eliminarse.
#df['release_date']=df['release_date'].dropna()
df = df.dropna(subset = ['release_date'])

In [108]:
df['release_date'].isnull().sum()

0

#### - Crear la columna 'release_year' donde extraerán el año de la fecha de estreno.

In [109]:
df.release_date

0        1995-10-30
1        1995-12-15
2        1995-12-22
3        1995-12-22
4        1995-02-10
            ...    
45460    1991-05-13
45462    2011-11-17
45463    2003-08-01
45464    1917-10-21
45465    2017-06-09
Name: release_date, Length: 45376, dtype: object

In [110]:
df.loc[19714]

belongs_to_collection      0
budget                   0.0
genres                     0
id                         0
original_language          0
overview                   0
popularity                 0
production_companies       0
production_countries       0
release_date               0
revenue                  0.0
runtime                  0.0
spoken_languages           0
status                     0
tagline                    0
title                      0
vote_average             0.0
vote_count               0.0
return                   NaN
Name: 19714, dtype: object

In [111]:
df.columns

Index(['belongs_to_collection', 'budget', 'genres', 'id', 'original_language',
       'overview', 'popularity', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'vote_average',
       'vote_count', 'return'],
      dtype='object')

In [112]:
# rellenamos los datos faltantes de la columna spoken_languages
df['spoken_languages']=df['spoken_languages'].fillna('Not Available')

#### - De haber fechas, deberán tener el formato AAAA-mm-dd, además deberán crear la columna release_year donde extraerán el año de la fecha de estreno.

In [113]:
# Cambiamos las fechas de la columna 'release_date' a formato AAAA-mm-dd, ignorando con 'coerce'
# los valores que no cumplen con el criterio para la funcion .to_datetime
df["release_date"] = pd.to_datetime(df['release_date'],errors='coerce')


In [114]:
df.release_date

0       1995-10-30
1       1995-12-15
2       1995-12-22
3       1995-12-22
4       1995-02-10
           ...    
45460   1991-05-13
45462   2011-11-17
45463   2003-08-01
45464   1917-10-21
45465   2017-06-09
Name: release_date, Length: 45376, dtype: datetime64[ns]

In [115]:
# Crear una nueva columna 'year' con el año extraído
df['release_year'] = df['release_date'].dt.year.fillna(0).astype(int)


In [116]:
# Crear una nueva columna 'release_month' con el año extraído
df['release_month'] = df['release_date'].dt.month.fillna(0).astype(int)


In [117]:
df.release_month

0        10
1        12
2        12
3        12
4         2
         ..
45460     5
45462    11
45463     8
45464    10
45465     6
Name: release_month, Length: 45376, dtype: int32

In [118]:
from datetime import datetime
import calendar

In [119]:
# Obtener el nombre del día de la semana en inglés
df['release_day'] = df['release_date'].dt.day_name()

In [120]:
# Rellenar los valores faltantes con una cadena vacía ('')
df['release_day'] = df['release_day'].fillna('')

In [121]:
df.release_day

0          Monday
1          Friday
2          Friday
3          Friday
4          Friday
           ...   
45460      Monday
45462    Thursday
45463      Friday
45464      Sunday
45465      Friday
Name: release_day, Length: 45376, dtype: object

In [122]:
#df['spoken_languages'][0:]
#ejemplo=pd.DataFrame(df['spoken_languages'])

In [123]:
# ejemplo para dividir con coma','  la cadena de texto en donde aparece un espacio. '\s'
#re.split('\s',df['spoken_languages'][0])

In [124]:
#valor_iso= df['spoken_languages'][1]
#print(re.search(r"'\w\w'",valor_iso))
# verificamos que los valores de la columna son cadena string.
#type(df['spoken_languages'][1])
#df_lenguaje= pd.DataFrame(df['spoken_languages'])
# alternativa para separar en nuevas columnas teniendo en cuenta como separador de la cadena string
# la coma ','
#df_lenguaje['spoken_languages'].str.split(',',n=3,expand=True)

In [125]:
df['spoken_languages'].unique()

array(["[{'iso_639_1': 'en', 'name': 'English'}]",
       "[{'iso_639_1': 'en', 'name': 'English'}, {'iso_639_1': 'fr', 'name': 'Français'}]",
       "[{'iso_639_1': 'en', 'name': 'English'}, {'iso_639_1': 'es', 'name': 'Español'}]",
       ...,
       "[{'iso_639_1': 'sv', 'name': 'svenska'}, {'iso_639_1': 'de', 'name': 'Deutsch'}]",
       "[{'iso_639_1': 'ar', 'name': 'العربية'}, {'iso_639_1': 'pl', 'name': 'Polski'}]",
       "[{'iso_639_1': 'ff', 'name': 'Fulfulde'}, {'iso_639_1': 'en', 'name': 'English'}]"],
      dtype=object)

In [126]:
import ast

def parse_spoken_languages(x):
    try:
        return ast.literal_eval(x)
    except (SyntaxError, ValueError):
        return []

df['spoken_languages'] = df['spoken_languages'].apply(lambda x: parse_spoken_languages(x) if pd.notnull(x) else [])

In [127]:
df['spoken_languages'] = df['spoken_languages'].apply( lambda x: [lang['name'] for lang in x] if isinstance(x,list) else x)
df.spoken_languages

0                  [English]
1        [English, Français]
2                  [English]
3                  [English]
4                  [English]
                ...         
45460              [English]
45462                     []
45463              [English]
45464                     []
45465              [English]
Name: spoken_languages, Length: 45376, dtype: object

In [128]:
# Otras opciones para parsear con regex
#df['iso_spoken_languages']=df.spoken_languages.str.extract("('\w\w')")
# Eliminar las comillas usando .str.strip()
#df['iso_spoken_languages'] = df['iso_spoken_languages'].str.strip("'")

In [129]:
df.spoken_languages

0                  [English]
1        [English, Français]
2                  [English]
3                  [English]
4                  [English]
                ...         
45460              [English]
45462                     []
45463              [English]
45464                     []
45465              [English]
Name: spoken_languages, Length: 45376, dtype: object

In [130]:
df.columns

Index(['belongs_to_collection', 'budget', 'genres', 'id', 'original_language',
       'overview', 'popularity', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'vote_average',
       'vote_count', 'return', 'release_year', 'release_month', 'release_day'],
      dtype='object')

In [131]:
df.isnull().sum()

belongs_to_collection    40885
budget                       0
genres                       0
id                           0
original_language           11
overview                   941
popularity                   0
production_companies         0
production_countries         0
release_date                 3
revenue                      0
runtime                    246
spoken_languages             0
status                      80
tagline                  24976
title                        0
vote_average                 0
vote_count                   0
return                   34464
release_year                 0
release_month                0
release_day                  0
dtype: int64

#### - Teniendo en cuenta las funciones a desarrollar, las siguientes columnas se consideran irrelevantes, ya sea por la falta de informacion o poco peso a la hora de evaluar las funciones. Las columnas son: 'belongs_to_collection', 'status', 'tagline', 'overview'.

In [132]:
df.drop(['belongs_to_collection','status','tagline'],axis=1,inplace=True)

In [133]:
df.columns

Index(['budget', 'genres', 'id', 'original_language', 'overview', 'popularity',
       'production_companies', 'production_countries', 'release_date',
       'revenue', 'runtime', 'spoken_languages', 'title', 'vote_average',
       'vote_count', 'return', 'release_year', 'release_month', 'release_day'],
      dtype='object')

In [134]:
df.genres.unique()

array(["[{'id': 16, 'name': 'Animation'}, {'id': 35, 'name': 'Comedy'}, {'id': 10751, 'name': 'Family'}]",
       "[{'id': 12, 'name': 'Adventure'}, {'id': 14, 'name': 'Fantasy'}, {'id': 10751, 'name': 'Family'}]",
       "[{'id': 10749, 'name': 'Romance'}, {'id': 35, 'name': 'Comedy'}]",
       ...,
       "[{'id': 80, 'name': 'Crime'}, {'id': 35, 'name': 'Comedy'}, {'id': 28, 'name': 'Action'}, {'id': 10751, 'name': 'Family'}]",
       "[{'id': 28, 'name': 'Action'}, {'id': 9648, 'name': 'Mystery'}, {'id': 53, 'name': 'Thriller'}, {'id': 27, 'name': 'Horror'}]",
       "[{'id': 10751, 'name': 'Family'}, {'id': 16, 'name': 'Animation'}, {'id': 10749, 'name': 'Romance'}, {'id': 35, 'name': 'Comedy'}]"],
      dtype=object)

In [135]:
import ast

def parse_genres(x):
    try:
        return ast.literal_eval(x)
    except (SyntaxError, ValueError):
        return []

df['genres'] = df['genres'].apply(lambda x: parse_genres(x) if pd.notnull(x) else [])

# hemos creado una función parse_genres que intenta evaluar la cadena de texto utilizando ast.literal_eval(). 
# Si ocurre algún error durante la evaluación, como una sintaxis incorrecta o un valor no válido, la función 
# capturará la excepción y retornará una lista vacía en su lugar.
# Luego, aplicamos esta función a cada valor de la columna 'genres' utilizando apply(). 
# Si el valor es NaN, se asigna una lista vacía directamente.
# Con este enfoque, cualquier valor incorrecto o incompatible en la columna 'genres' se manejará adecuadamente 
# y se asignará una lista vacía en su lugar. Esto debería evitar que se produzca el error 
# "ValueError: malformed node or string: 0".


In [136]:
df['genres'] = df['genres'].apply( lambda x: [lang['name'] for lang in x] if isinstance(x,list) else x)
df.genres

0         [Animation, Comedy, Family]
1        [Adventure, Fantasy, Family]
2                   [Romance, Comedy]
3            [Comedy, Drama, Romance]
4                            [Comedy]
                     ...             
45460        [Drama, Action, Romance]
45462                         [Drama]
45463       [Action, Drama, Thriller]
45464                              []
45465                              []
Name: genres, Length: 45376, dtype: object

In [137]:
df.columns

Index(['budget', 'genres', 'id', 'original_language', 'overview', 'popularity',
       'production_companies', 'production_countries', 'release_date',
       'revenue', 'runtime', 'spoken_languages', 'title', 'vote_average',
       'vote_count', 'return', 'release_year', 'release_month', 'release_day'],
      dtype='object')

In [138]:
# Utilizamos el mismo metodo que 'genres' (para columnas anidadas)
def parse_production_companies(x):
    try:
        return ast.literal_eval(x)
    except (SyntaxError, ValueError):
        return []

df['production_companies'] = df['production_companies'].apply(lambda x: parse_production_companies(x) if pd.notnull(x) else [])

In [139]:
df['production_companies'] = df['production_companies'].apply( lambda x: [lang['name'] for lang in x] if isinstance(x,list) else x)
df.production_companies

0                                [Pixar Animation Studios]
1        [TriStar Pictures, Teitler Film, Interscope Co...
2                           [Warner Bros., Lancaster Gate]
3                 [Twentieth Century Fox Film Corporation]
4             [Sandollar Productions, Touchstone Pictures]
                               ...                        
45460    [Westdeutscher Rundfunk (WDR), Working Title F...
45462                                        [Sine Olivia]
45463                            [American World Pictures]
45464                                          [Yermoliev]
45465                                                   []
Name: production_companies, Length: 45376, dtype: object

In [140]:
# Utilizamos el mismo metodo que 'genres' (para columnas anidadas)
def parse_production_countries(x):
    try:
        return ast.literal_eval(x)
    except (SyntaxError, ValueError):
        return []

df['production_countries'] = df['production_countries'].apply(lambda x: parse_production_countries(x) if pd.notnull(x) else [])

In [141]:
df['production_countries'] = df['production_countries'].apply( lambda x: [lang['iso_3166_1'] for lang in x] if isinstance(x,list) else x)
df.production_countries

0                    [US]
1                    [US]
2                    [US]
3                    [US]
4                    [US]
               ...       
45460    [CA, DE, GB, US]
45462                [PH]
45463                [US]
45464                [RU]
45465                [GB]
Name: production_countries, Length: 45376, dtype: object

In [142]:
df_v1=df

In [143]:
df_v1.columns

Index(['budget', 'genres', 'id', 'original_language', 'overview', 'popularity',
       'production_companies', 'production_countries', 'release_date',
       'revenue', 'runtime', 'spoken_languages', 'title', 'vote_average',
       'vote_count', 'return', 'release_year', 'release_month', 'release_day'],
      dtype='object')

In [144]:
# opcion 1 -Pasamos los codigoas a uppercase. utilizamos el método str.upper() para convertir los códigos ISO a mayúsculas y el 
# método tolist()  para convertir la columna en una lista.
df_v1['original_language']=df_v1['original_language'].str.upper().tolist()
# opcion 2 -Convertir los códigos ISO a mayúsculas y agregar corchetes
#df['original_language'] = df['original_language'].apply(lambda x: f"[{x.upper()}]")


In [145]:
# Pasamos float a int
df_v1['vote_count']=df_v1.vote_count.astype(int)

In [146]:
# Pasamos object to int
df_v1['id']=df_v1.id.astype(int)

In [147]:
df_v1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 45376 entries, 0 to 45465
Data columns (total 19 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   budget                45376 non-null  float64       
 1   genres                45376 non-null  object        
 2   id                    45376 non-null  int32         
 3   original_language     45362 non-null  object        
 4   overview              44435 non-null  object        
 5   popularity            45376 non-null  object        
 6   production_companies  45376 non-null  object        
 7   production_countries  45376 non-null  object        
 8   release_date          45373 non-null  datetime64[ns]
 9   revenue               45376 non-null  float64       
 10  runtime               45130 non-null  float64       
 11  spoken_languages      45376 non-null  object        
 12  title                 45376 non-null  object        
 13  vote_average         

In [148]:
df_v1['release_date'].fillna('1900-01-01',inplace=True)

In [149]:
df_v1 = df_v1.drop_duplicates(subset='id')

In [150]:
df_v1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 45344 entries, 0 to 45465
Data columns (total 19 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   budget                45344 non-null  float64       
 1   genres                45344 non-null  object        
 2   id                    45344 non-null  int32         
 3   original_language     45332 non-null  object        
 4   overview              44403 non-null  object        
 5   popularity            45344 non-null  object        
 6   production_companies  45344 non-null  object        
 7   production_countries  45344 non-null  object        
 8   release_date          45344 non-null  datetime64[ns]
 9   revenue               45344 non-null  float64       
 10  runtime               45098 non-null  float64       
 11  spoken_languages      45344 non-null  object        
 12  title                 45344 non-null  object        
 13  vote_average         

In [151]:
df_v1.to_csv('ETL_v2_Movies.csv',index=False)

In [152]:
df_ML = df_v1[['id','overview','title','genres','original_language','spoken_languages','popularity','production_companies','production_countries','release_year','vote_count','vote_average']]

In [153]:
df_ML['original_language'].fillna('XX',inplace=True)

C:\Users\TheFercho\AppData\Local\Temp\ipykernel_9928\1004109691.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ML['original_language'].fillna('XX',inplace=True)


In [154]:
df_ML['overview'].fillna('No info',inplace=True)

C:\Users\TheFercho\AppData\Local\Temp\ipykernel_9928\2504503399.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ML['overview'].fillna('No info',inplace=True)


In [155]:
# Convertir la columna 'genres' en valores separados por comas
df_ML['genres'] = df_ML['genres'].apply(lambda x: ', '.join(x))

C:\Users\TheFercho\AppData\Local\Temp\ipykernel_9928\2663302485.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ML['genres'] = df_ML['genres'].apply(lambda x: ', '.join(x))


In [156]:
#df_ML.to_csv('overview_ML_dataset.csv',index=False)

In [175]:
df_ML.info()

<class 'pandas.core.frame.DataFrame'>
Index: 45344 entries, 0 to 45465
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id                    45344 non-null  int32  
 1   title                 45344 non-null  object 
 2   genres                45344 non-null  object 
 3   original_language     45344 non-null  object 
 4   spoken_languages      45344 non-null  object 
 5   popularity            45344 non-null  object 
 6   production_companies  45344 non-null  object 
 7   production_countries  45344 non-null  object 
 8   release_year          45344 non-null  int32  
 9   vote_count            45344 non-null  int32  
 10  vote_average          45344 non-null  float64
dtypes: float64(1), int32(3), object(7)
memory usage: 3.6+ MB


In [176]:
df_ML.genres

0         Animation, Comedy, Family
1        Adventure, Fantasy, Family
2                   Romance, Comedy
3            Comedy, Drama, Romance
4                            Comedy
                    ...            
45460        Drama, Action, Romance
45462                         Drama
45463       Action, Drama, Thriller
45464                              
45465                              
Name: genres, Length: 45344, dtype: object

In [163]:
df_ML.genres.unique()

array(['Animation, Comedy, Family', 'Adventure, Fantasy, Family',
       'Romance, Comedy', ..., 'Crime, Comedy, Action, Family',
       'Action, Mystery, Thriller, Horror',
       'Family, Animation, Romance, Comedy'], dtype=object)

In [177]:
# Obtener una lista de todos los géneros en el DataFrame
all_genres = set([genre for sublist in df['genres'].str.split(', ') for genre in sublist])

TypeError: 'float' object is not iterable

In [ ]:
from sklearn.preprocessing import OneHotEncoder

# Crear una instancia de OneHotEncoder y especificar los géneros más relevantes
top_genres = ['Action', 'Adventure', 'Comedy', 'Drama', 'Fantasy', 'Romance', 'Sci-Fi', 'Thriller', 'Family', 'Animation']
encoder = OneHotEncoder(categories=[top_genres])

# Ajustar y transformar los datos
encoded_data = encoder.fit_transform(df_ML[['genres']])

# Crear un nuevo DataFrame con las columnas codificadas
encoded_df_ML = pd.DataFrame(encoded_data.toarray(), columns=encoder.get_feature_names_out(['genres']))

# Concatenar el nuevo DataFrame codificado con el DataFrame original
df_ML_encoded = pd.concat([df_ML, encoded_df_ML], axis=1)

In [ ]:
#df_ML_encoded=df_ML[['id','overview','title','genres','original_language','spoken_languages','popularity','production_companies','production_countries','release_year','vote_count','vote_average']]

In [168]:
df_ML.drop(columns='overview',inplace=True)

C:\Users\TheFercho\AppData\Local\Temp\ipykernel_9928\925467467.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ML.drop(columns='overview',inplace=True)


In [179]:
df_ML.to_csv('ML_dataset.csv',index=False)

In [178]:
df_ML.columns

Index(['id', 'title', 'genres', 'original_language', 'spoken_languages',
       'popularity', 'production_companies', 'production_countries',
       'release_year', 'vote_count', 'vote_average'],
      dtype='object')